In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from konlpy.tag import Okt
from gensim.models import Word2Vec
from scipy.spatial import distance

## 전처리

In [7]:
raw_df = pd.read_csv('/Users/space/Google Drive/5 Programming/Project_ThematicInvest/Data/data_for_use/20200629_news_data.csv',
                     index_col=0, encoding = 'utf-8')
raw_df.dropna(inplace=True)

In [8]:
raw_df

,news
0,통신사 관계 없이 이용 가능 24개월 인수형 장기렌탈 서울 뉴시스 오동현 기자 KT...
1,20년 된 전자금융거래법 정비돼야 전자금융 모든 거래 총괄하게 되면 내년 비대면 상...
2,구내식당 모바일스루 서비스…7월 3곳 시범 적용 서울 뉴시스 오동현 기자 모바일 전...
3,소니가 프리미엄 디지털카메라로 분류한 RX100 제품군은 등장 초기에는 존재감이 분...
4,법원 행정처분 집행 정지 인용 지디넷코리아 손예술 기자 법원이 하나금융지주 함영주 ...
...,...
19938,SNS에 부동산정책 실패 강력 비판 “참모로부터 잘못된 신화 학습 큰일 진보 경제학...
19939,김선혁 교수 한국정치 혁신 촉구 인류 대 바이러스 절체절명 위기 정파간 갈등·대립 ...
19940,4·15 총선서 이미 SNS 유세 등장 합당 온라인투표 공천 화상면접도 돈·조직 기...
19941,여야 합의점 못 찾았지만 29일 오전 10시 다시 회동 법사위 절충안 마련 등 주목...


In [10]:
raw_df = raw_df[['Theme','Article']]
raw_df['Article'] = raw_df['Article'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
raw_df.head()

,Theme,Article
0,방탄소년단,더팩트 방탄소년단 티머니카드 단독 판매편의점 가 내달 일부터 전국 점포에서 방탄소...
1,방탄소년단,뉴스 방탄소년단 티머니카드 출시만장 한정 판매정규 집 콘셉트 앨범으로 디자인내달 일...
2,방탄소년단,방탄소년단 티머니카드 단독 판매글로벌 슈퍼스타 방탄소년단이 올해도 티머니에 쏙...
3,방탄소년단,조선비즈칠성사이다에 방탄소년단이 복숭아청귤 스페셜 에디션롯데칠성음료는 방탄소년단 ...
4,방탄소년단,뉴시스방탄소년단 담았다칠성사이다 에디션 출시서울뉴시스 최지윤 기자 롯데칠성음료는...


In [6]:
# tokenize data

tokenizer = Mecab()

raw_tokenized_data = []
for content in tqdm(raw_df['Article']):
    token = tokenizer.morphs(content)
    stop_words = ['으로', '로도', '지만', '에서', '려는', '하다']
    for word in token:
        if len(word) == 1:
            stop_words.append(word)
    final_tokens = [word for word in token if word not in stop_words]
    raw_tokenized_data.append(final_tokens)

100%|██████████| 28200/28200 [01:30<00:00, 313.06it/s]


In [25]:
# make train_data

train_data = raw_df.copy()
train_data['tokens'] = pd.Series(raw_tokenized_data)

In [9]:
train_data.head()

,Theme,Article,tokens
0,방탄소년단,더팩트 방탄소년단 티머니카드 단독 판매편의점 가 내달 일부터 전국 점포에서 방탄소...,"[방탄소년단, 티머니, 카드, 단독, 판매, 편의점, 내달, 부터, 전국, 점포, ..."
1,방탄소년단,뉴스 방탄소년단 티머니카드 출시만장 한정 판매정규 집 콘셉트 앨범으로 디자인내달 일...,"[뉴스, 방탄소년단, 티머니, 카드, 출시, 한정, 판매, 정규, 콘셉트, 앨범, ..."
2,방탄소년단,방탄소년단 티머니카드 단독 판매글로벌 슈퍼스타 방탄소년단이 올해도 티머니에 쏙...,"[방탄소년단, 티머니, 카드, 단독, 판매, 글로벌, 슈퍼스타, 방탄소년단, 올해,..."
3,방탄소년단,조선비즈칠성사이다에 방탄소년단이 복숭아청귤 스페셜 에디션롯데칠성음료는 방탄소년단 ...,"[조선, 비즈, 칠성, 사이다, 방탄소년단, 복숭아, 청귤, 스페셜, 에디션, 롯데..."
4,방탄소년단,뉴시스방탄소년단 담았다칠성사이다 에디션 출시서울뉴시스 최지윤 기자 롯데칠성음료는...,"[뉴시스, 방탄소년단, 칠성사, 에디션, 출시, 서울, 뉴시스, 최지윤, 기자, 롯..."


## Model Train

In [11]:
news_tokens = train_data['tokens'].tolist()  # data input as list

# params
v_dimension = 300
v_window = 8

model = Word2Vec(sentences = news_tokens, size = v_dimension, window = v_window, min_count = 5, workers = 4, sg = 0)

In [12]:
model.wv.vectors.shape

(38552, 300)

In [13]:
print(model.wv.most_similar("아이폰"))

[('아이패드', 0.7444136142730713), ('애플', 0.7387940883636475), ('어댑터', 0.619560182094574), ('안드로이드', 0.6110464930534363), ('스냅드래곤', 0.5927270650863647), ('서피스', 0.5812198519706726), ('스크린', 0.574851930141449), ('라이트닝', 0.5703201293945312), ('맥북', 0.5664032101631165), ('블랙베리', 0.5628055334091187)]


In [14]:
# sentence vectors without normalization

def without_normal(tokens):
    vectors = []
    for token in tokens:
        init_v = np.array([0.0]*v_dimension)
        for word in token:
            word_vectors = model.wv
            if word in word_vectors.vocab:
                v = model.wv[word]
                init_v = init_v + v
        vectors.append(init_v)
    
    frame = { 'themes': train_data['Theme'].tolist(), 'vectors': vectors }
    result = pd.DataFrame(frame) 
    
    return result

In [15]:
# theme vectors without normalization

def theme_without_normal(news_df):
    theme_list = []
    vector_list = []
    for theme in news_df['themes'].unique():
        temp_df = news_df.loc[news_df['themes'] == theme]
        add_v = np.array([0.0]*v_dimension)
        for vec in temp_df['vectors']:
            add_v  = add_v + vec
        theme_list.append(theme)
        vector_list.append(add_v)
        
    frame = { 'themes': theme_list, 'vectors': vector_list }
    result = pd.DataFrame(frame)
    
    return result

In [16]:
news_vectors_df = without_normal(train_data['tokens'])

In [17]:
news_vectors_df.head()

,themes,vectors
0,방탄소년단,"[81.83884788467549, 14.1695456225425, -44.6049..."
1,방탄소년단,"[79.30194197152741, 39.09123660530895, -40.572..."
2,방탄소년단,"[81.88316881936044, 27.98129623476416, -54.289..."
3,방탄소년단,"[67.23731970321387, 8.51590171258431, -27.9272..."
4,방탄소년단,"[45.1811446743086, 5.173289151745848, -5.27254..."


In [18]:
theme_vectors_df = theme_without_normal(news_vectors_df)

In [19]:
theme_vectors_df.head()

,themes,vectors
0,방탄소년단,"[5746.59946695203, -2133.1955969270784, -5507...."
1,5G,"[3760.2759751131816, 1089.4813753533235, 15035..."
2,보톡스,"[-4661.68236238329, 2435.5982249401786, 6313.9..."
3,마스크,"[-2542.781907488039, 4610.864502544882, 4371.8..."
4,원격진료,"[-6637.810228657618, 2384.9900262603187, 21742..."


In [26]:
theme_vectors_df.to_csv('theme_vectors_xnorm')

## Test

In [20]:
# news vectors without normalization

def vectorize_without_normal(news):
    news_words = news.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    token = tokenizer.morphs(news)
    stop_words = ['으로', '로도', '지만', '에서', '려는', '하다']
    for word in token:
        if len(word) == 1:
            stop_words.append(word)
    final_tokens = [word for word in token if word not in stop_words]
    init_v = np.array([0.0]*v_dimension)
    for word in final_tokens:
        word_vectors = model.wv
        if word in word_vectors.vocab:
            v = model.wv[word]
            init_v = init_v + v
    return init_v

In [21]:
# test

def test(news):
    news_vec = vectorize_without_normal(news)
    result = []
    for theme in theme_vectors_df['vectors']:
        cosine = 1 - distance.cosine(theme, news_vec)
        result.append(cosine)
    new_df = pd.DataFrame(data=np.zeros([282,2]), columns=['Theme', 'Result'])
    new_df['Theme'] = theme_vectors_df['themes']
    new_df['Result'] = result

    new_df.sort_values('Result', ascending=False, inplace=True)
    return new_df
    

In [22]:
test_news = "미국의 최대 새해맞이 행사죠, '뉴 이어스 로킹 이브'에 방탄소년단이 출연해 월드 스타로서의 면모를 과시했습니다. 지난해 빌보드 순위와 월드 투어에서 괄목할 만한 성장을 거둔 방탄소년단, 올해도 한류를 이끌 것으로 기대되고 있습니다. 김혜은 기자입니다. 방탄소년단, BTS는 미국 뉴욕의 심장인 타임스퀘어에서 새해를 맞았습니다. 미국의 최대 새해맞이 행사인 '뉴 이어스 로킹 이브'에 참가한 것입니다. 포스트 말론, 샘 헌트 등 세계적인 가수들과 어깨를 나란히 하며 미국 안방에 스며들었습니다. [리안나 제이콥슨 / BTS 팬 : BTS가 사랑받고 있다는 것, 그들이 놀랍고 재능 있고 매 순간 가치 있다는 것을 알려줘야 합니다.] BTS는 지난해 '빌보드 200' 1위와 아메리칸 뮤직 어워즈 3관왕 등 괄목할 기록을 낳았습니다. 한국 가수 최초로 전 세계 스타디움 투어도 성공적으로 마무리했습니다. 투어의 대미를 장식한 서울 공연에서만 사흘 동안 무려 1조 원에 육박하는 경제 효과를 낳은 것으로 집계됐습니다. 공연으로 18만 명 넘는 외국인이 우리나라를 방문했는데, 평창동계올림픽 당시 외국인 방문객의 67%에 해당하는 수치입니다. BTS의 올해 행보에 더욱 기대가 쏠리는 대목입니다. 지난달 공개한 티저 사진으로 올 초 예정된 새 앨범과 투어에 대한 기대감이 올라갔습니다. [지민 / 방탄소년단 멤버 (지난달 MAMA 시상식) : 여러분들이 기대하시는 것보다 훨씬 더 좋은 앨범으로 저희가 여러분들에게 나타날 수 있을 것 같아요.] BTS 이후 '빌보드200' 1위를 차지한 슈퍼엠, 미국 프로그램에 자주 등장하는 몬스타엑스와 NCT 127 등 K팝의 지형은 갈수록 확대되고 있습니다. [김헌식 / 대중문화평론가 : 음악의 유통구조 자체가 유튜브를 포함한 SNS를 중심으로 확산하고 있기 때문에 여기에서 계속 추이를 따라가는 수준이라면 10년 이상까지도 가능할 수 있다는 거죠.] 새로운 10년이 시작되는 2020년, BTS가 미국 심장부인 뉴욕에서 새해를 시작한 것도, 한류의 새로운 10년에 대한 상징으로도 해석되고 있습니다."

In [23]:
result = test(test_news)

In [24]:
result.head()

,Theme,Result
7,기생충,0.777822
83,영화,0.663112
0,방탄소년단,0.656203
257,아기상어,0.647218
118,광고,0.593024
